# IPs únicos históricos por API

In [1]:
# setup
import os
import sys

# para importar módulos en '../../tools'
module_path = os.path.abspath(os.path.join('../../tools'))
if module_path not in sys.path:
    sys.path.append(module_path)


In [2]:
import locale

import glob
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.patches as patches
import seaborn as sns

import json

# from tools.download import download

import analytics_tools

from IPython.core.display import display, HTML

locale.setlocale(locale.LC_ALL, '')

pd.set_option('precision',2)
pd.set_option('display.float_format', lambda x: locale.format_string('%.2f', x, 1))


In [3]:
# formarters
f_ar = lambda x: locale.format_string('%.2f', x, 1)
d_ar = lambda x: locale.format_string('%d', x, 1)

tz_arg = lambda x: pd.to_datetime(x).tz_localize('UTC').tz_convert(tz='America/Argentina/Buenos_Aires')

TABLE_COUNTER = 0
TABLE_TEMPLATE = """
<center><strong><small>{title}</small></strong></center>
<center>{table}</center>
<center><strong><small>Tabla {table_number}</small></strong></center>
"""

def add_totals(df):
    df_to_append = pd.DataFrame(df.sum(numeric_only=True)).T
    
    df_appended = df.append(df_to_append)
    
    as_list = df_appended.index.tolist()
    idx = as_list.index(0)
    as_list[idx] = 'Total'
    df_appended.index = as_list
    
    return df_appended

def table_counter():
    global TABLE_COUNTER
    TABLE_COUNTER += 1
    return TABLE_COUNTER

def add_style_to_df(df, subset, color='black', font_weight=None):
    render = df.style.set_properties(
        subset= subset, 
        **{'font-weight': font_weight, 'color':color}).render().replace('\n','')
    return render

def add_title(df_html, title):
    str_table = TABLE_TEMPLATE.format(title=title, table=df_html, table_number=table_counter())
    return str_table.replace('\n','')

def put_df_on_report(df, title, subset=None, color='black', font_weight='bold'):
    if subset:
        df_html = add_style_to_df(df, subset=subset,color=color, font_weight=font_weight)
    else:
        df_html = df.to_html().replace('\n','')
    return add_title(df_html, title)

In [4]:
def read_files_to_df(directory):
    """Lee CSVs de misma estructura en un directorio a un solo DataFrame."""

    file_pattern = os.path.join(directory, "*.csv")
    dfs = [pd.read_csv(file, encoding="utf8", parse_dates=True)
           for file in glob.glob(file_pattern)]
        
    return pd.concat(dfs, axis=0)

def count_file_row(directory):
    """Lee CSVs de misma estructura en un directorio a un solo DataFrame."""
    
    days_list = []
    file_pattern = os.path.join(directory, "*.csv")
    
    for file in glob.glob(file_pattern):
        day_df = pd.read_csv(file, encoding="utf8", parse_dates=True)
        r, c = day_df.shape
        day = file[-14:-4]
        days_list.append([day, r, file])
        
    return pd.DataFrame(days_list)

In [5]:
# actualizo los analytics de ambas apis
analytics_tools.update_analytics()

In [6]:
dir_sdt = analytics_tools.DIR_DATA_SDT
dir_gr = analytics_tools.DIR_DATA_GR

paths = [dir_sdt, dir_gr]
titles = ['Series de Tiempo', 'Georef']
apis = ['series-tiempo', 'georef']

htmls = []

for path, title, api in zip(paths,titles,apis):
    
    df_analytics = read_files_to_df(path)

    df_analytics['start_time_utc'] = pd.to_datetime(df_analytics['start_time'])
#     df_analytics['start_time'] = pd.DatetimeIndex(df_analytics['start_time']).tz_localize('UTC').tz_convert(tz='America/Argentina/Buenos_Aires')
    df_analytics['start_time'] = pd.DatetimeIndex(df_analytics['start_time']).tz_convert(tz='America/Argentina/Buenos_Aires')
    df_analytics["date_full"] = pd.DatetimeIndex(df_analytics.start_time).normalize()
    df_analytics["date"] = pd.DatetimeIndex(df_analytics.start_time).strftime('%Y-%m')

    end_date = str(df_analytics["date_full"].max())[:10]
    
    ips_birthdate = df_analytics[['ip_address','date']].groupby(['ip_address']).min()['date'].reset_index()
    ips_birthdate.rename({'date':'birthdate'}, axis=1, inplace=True)

    unique_ips_dates = df_analytics[['ip_address','date']].drop_duplicates()
    df_ips = unique_ips_dates.merge(ips_birthdate, on='ip_address', how='left')

    # cantidad de ips únicas
    ips_nunique = df_ips.ip_address.nunique()

    # cantidad de ips nuevos por mes
    ips_born_per_month = df_ips[['birthdate','ip_address']].groupby('birthdate').nunique()['ip_address'].reset_index()
    ips_born_per_month.rename({'ip_address':'ips_nuevos'}, axis=1, inplace=True)

    # cantidad de ips por mes
    ips_per_month = df_ips[['date','ip_address']].groupby('date').nunique()['ip_address'].reset_index()
    ips_per_month.rename({'ip_address':'ips_unicos'}, axis=1, inplace=True)

    # acumulado de ips únicas por mes
    ips_per_month['ips_acumulados'] = ips_per_month.ips_unicos.cumsum()

    df_agg_ips = ips_born_per_month.merge(ips_per_month, left_on='birthdate', right_on='date')
    df_agg_ips.rename({'date':''}, axis=1, inplace=True)
    df_agg_ips.set_index('', inplace=True)
    df_agg_ips.drop('birthdate', axis=1, inplace=True)

    df_agg_ips = add_totals(df_agg_ips)

    ips_recurrentes = (1 - df_agg_ips.ips_nuevos/df_agg_ips.ips_unicos)*100
    df_agg_ips['ips_recurrentes'] = ips_recurrentes.apply(lambda x: '{:.2f} %'.format(x))

    df_agg_ips.to_csv('../reports/ips-historicos-{}.csv'.format(api))
    
    htmls.append(put_df_on_report(df_agg_ips, '{} - Hasta el día {}'.format(title, end_date)))

/Users/abenassi/anaconda/envs/analytics/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


In [7]:
sdt = htmls[0]
gr = htmls[1]

[Descargar CSV - series de tiempo](https://datosgobar.github.io/analytics/api-gateway/reports/ips-historicos-series-tiempo.csv)

[Descargar CSV - georef](https://datosgobar.github.io/analytics/api-gateway/reports/ips-historicos-georef.csv)

In [8]:
display(HTML(sdt))

,ips_nuevos,ips_unicos,ips_acumulados,ips_recurrentes
2018-01,203,203,203,0.00 %
2018-02,201,265,468,24.15 %
2018-03,148,231,699,35.93 %
2018-04,33,39,738,15.38 %
2018-05,83,153,891,45.75 %
2018-06,219,325,1216,32.62 %
2018-07,164,249,1465,34.14 %
2018-08,384,485,1950,20.82 %
2018-09,1090,1261,3211,13.56 %
2018-10,931,1194,4405,22.03 %


In [9]:
display(HTML(gr))

,ips_nuevos,ips_unicos,ips_acumulados,ips_recurrentes
2018-07,24,24,24,0.00 %
2018-08,209,227,251,7.93 %
2018-09,422,487,738,13.35 %
2018-10,15350,15556,16294,1.32 %
2018-11,129650,133849,150143,3.14 %
2018-12,76930,98694,248837,22.05 %
2019-01,52015,74121,322958,29.82 %
2019-02,26507,36613,359571,27.60 %
2019-03,16225,23756,383327,31.70 %
2019-04,15951,23773,407100,32.90 %
